In [ ]:
# Loading the data

import pandas as pd
from pathlib import Path

# Define base folder
base_path = Path("C:/Users/20231737/OneDrive - TU Eindhoven/Raw Data for CBL/Full Data")  # Adjust for own storage logic

# List to store individual DataFrames
dfs = []

# Folders where cleaned files will be saved
output_path = Path("C:/Users/20231737/OneDrive/Dokumentumok/Y2Q4/CBL/Other Data") # Adjust for own storage logic
output_path.mkdir(exist_ok=True)

# Loop through each subfolder
for month_folder in base_path.iterdir():
    if month_folder.is_dir():
        monthly_dfs = [] # Collect data from both police forces

        for csv_file in month_folder.glob("*stop*.csv"):
            if "city-of-london" in csv_file.name or "metropolitan" in csv_file.name:
                try:
                    # Load CSV
                    df = pd.read_csv(csv_file)

                    # Drop duplicates
                    df = df.drop_duplicates()

                    monthly_dfs.append(df)

                except Exception as e:
                    print(f"Failed to process {csv_file}: {e}") 
        if monthly_dfs:
            df = pd.concat(monthly_dfs, ignore_index=True)
            dfs.append(df)

# Combine all into a single DataFrame
stop_df = pd.concat(dfs, ignore_index=True)
stop_spatial_df = pd.concat(dfs, ignore_index=True)

C:\Users\20231737\AppData\Local\Temp\ipykernel_29668\2086909638.py:35: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df = pd.concat(monthly_dfs, ignore_index=True)
C:\Users\20231737\AppData\Local\Temp\ipykernel_29668\2086909638.py:35: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df = pd.concat(monthly_dfs, ignore_index=True)
C:\Users\20231737\AppData\Local\Temp\ipykernel_29668\2086909638.py:35: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dty

In [ ]:
# Check size, head

print(len(stop_df))
print(len(stop_spatial_df))
print(stop_df.head())

1690491
1690491
                        Type                       Date  \
0              Person search  2015-03-02T16:40:00+00:00   
1              Person search  2015-03-02T16:40:00+00:00   
2              Person search  2015-03-02T18:45:00+00:00   
3              Person search  2015-03-02T19:15:00+00:00   
4  Person and Vehicle search  2015-03-03T15:50:00+00:00   

  Part of a policing operation  Policing operation  Latitude  Longitude  \
0                          NaN                 NaN       NaN        NaN   
1                          NaN                 NaN       NaN        NaN   
2                          NaN                 NaN       NaN        NaN   
3                          NaN                 NaN       NaN        NaN   
4                          NaN                 NaN       NaN        NaN   

  Gender Age range                          Self-defined ethnicity  \
0   Male     25-34       Asian or Asian British - Bangladeshi (A3)   
1   Male     25-34       Asian or Asia

In [78]:
# Null data check for dataset

# Count nulls per column
null_counts = stop_df.isnull().sum()

# Show columns with at least one null
print(null_counts[null_counts > 0])


Part of a policing operation                 191011
Policing operation                          1690491
Latitude                                     345175
Longitude                                    345175
Gender                                        19764
Age range                                    192719
Self-defined ethnicity                        21327
Officer-defined ethnicity                     34401
Legislation                                     159
Object of search                              86333
Outcome                                         246
Outcome linked to object of search          1607190
Removal of more than just outer clothing    1605624
dtype: int64


In [79]:
# List mainly empty columns' values

outcome = sorted(stop_df["Outcome linked to object of search"].unique())
removal = sorted(stop_df["Removal of more than just outer clothing"].unique())
part = sorted(stop_df["Part of a policing operation"].unique())
print("Outcomes:")
for i in outcome:
    print(i)
print("Removal:")
for i in removal:
    print(i)
print("Part:")
for i in part:
    print(i)

Outcomes:
False
True
nan
Removal:
False
True
nan
Part:
nan
0.0


In [80]:
# Police op check

notpl = stop_df[stop_df["Part of a policing operation"] == 0.]
print(len(notpl))

1499480


In [81]:
# Cleaning of normal

# Location outlier removal (null kept)

boundary_mask = ((stop_df["Latitude"] <= 51.69) & (stop_df["Latitude"] >= 51.28) &
                 (stop_df["Longitude"] <= 0.33) & (stop_df["Longitude"] >= -0.51))

# Combine with mask that keeps NaNs
stop_df = stop_df[boundary_mask | stop_df["Latitude"].isna() | stop_df["Longitude"].isna()]

# Unnecessary column removal (Part of a policing operation, Policing operation, Self-defined ethnicity, Officer-defined ethnicity)

if "Part of a policing operation" in stop_df.columns:
    stop_df = stop_df.drop(columns=["Part of a policing operation"])
if "Policing operation" in stop_df.columns and stop_df["Policing operation"].isna().all():
    stop_df = stop_df.drop(columns=["Policing operation"])
if "Self-defined ethnicity" in stop_df.columns:
    stop_df = stop_df.drop(columns=["Self-defined ethnicity"])
if "Officer-defined ethnicity" in stop_df.columns:
    stop_df = stop_df.drop(columns=["Officer-defined ethnicity"])

# Cleaning of spatial

# Location outlier removal (null removed)

stop_spatial_df = stop_spatial_df[(stop_spatial_df["Latitude"] <= 51.69) & 
                          (stop_spatial_df["Latitude"] >= 51.28) &
                          (stop_spatial_df["Longitude"] <= 0.33) &
                          (stop_spatial_df["Longitude"] >= -0.51)]

# Unnecessary column removal (Part of a policing operation, Policing operation, Self-defined ethnicity, Officer-defined ethnicity)

if "Part of a policing operation" in stop_spatial_df.columns:
    stop_spatial_df = stop_spatial_df.drop(columns=["Part of a policing operation"])
if "Policing operation" in stop_spatial_df.columns and stop_spatial_df["Policing operation"].isna().all():
    stop_spatial_df = stop_spatial_df.drop(columns=["Policing operation"])
if "Self-defined ethnicity" in stop_spatial_df.columns:
    stop_spatial_df = stop_spatial_df.drop(columns=["Self-defined ethnicity"])
if "Officer-defined ethnicity" in stop_spatial_df.columns:
    stop_spatial_df = stop_spatial_df.drop(columns=["Officer-defined ethnicity"])


In [82]:
# Outliers gone check

outofboundary_df = stop_df[(stop_df["Latitude"] > 51.69) | 
                          (stop_df["Latitude"] < 51.28) |
                          (stop_df["Longitude"] > 0.33) |
                          (stop_df["Longitude"] < -0.51)]

outofboundary_spatial_df = stop_spatial_df[(stop_spatial_df["Latitude"] > 51.69) | 
                          (stop_spatial_df["Latitude"] < 51.28) |
                          (stop_spatial_df["Longitude"] > 0.33) |
                          (stop_spatial_df["Longitude"] < -0.51)]

print(len(outofboundary_df))
print(len(outofboundary_spatial_df))

0
0


In [83]:
# After cleaning check of data

print(len(stop_df))
print(len(stop_spatial_df))
print(stop_df.head())

1690262
1345087
                        Type                       Date  Latitude  Longitude  \
0              Person search  2015-03-02T16:40:00+00:00       NaN        NaN   
1              Person search  2015-03-02T16:40:00+00:00       NaN        NaN   
2              Person search  2015-03-02T18:45:00+00:00       NaN        NaN   
3              Person search  2015-03-02T19:15:00+00:00       NaN        NaN   
4  Person and Vehicle search  2015-03-03T15:50:00+00:00       NaN        NaN   

  Gender Age range                                        Legislation  \
0   Male     25-34  Police and Criminal Evidence Act 1984 (section 1)   
1   Male     25-34  Police and Criminal Evidence Act 1984 (section 1)   
2   Male     25-34  Police and Criminal Evidence Act 1984 (section 1)   
3   Male   over 34  Police and Criminal Evidence Act 1984 (section 1)   
4   Male     25-34  Police and Criminal Evidence Act 1984 (section 1)   

  Object of search           Outcome Outcome linked to object of

In [ ]:
# Count nulls per column after cleaning

null_counts_new = stop_df.isnull().sum()
print("Normal:")
print(null_counts_new)

null_counts_new_spatial = stop_spatial_df.isnull().sum()
print("\nSpatial:")
print(null_counts_new_spatial)

Normal:
Type                                              0
Date                                              0
Latitude                                     345175
Longitude                                    345175
Gender                                        19755
Age range                                    192689
Legislation                                     159
Object of search                              86320
Outcome                                         241
Outcome linked to object of search          1607085
Removal of more than just outer clothing    1605519
dtype: int64

Spatial:
Type                                              0
Date                                              0
Latitude                                          0
Longitude                                         0
Gender                                        16240
Age range                                    146984
Legislation                                      99
Object of search                 

In [85]:
# Data for mostly empty columns

# Normal
print("Normal dataset:")

link_true = stop_df[stop_df["Outcome linked to object of search"] == True]
print("link true:", len(link_true))

link_false = stop_df[stop_df["Outcome linked to object of search"] == False]
print("link false:", len(link_false))

removal_true = stop_df[stop_df["Removal of more than just outer clothing"] == True]
print("removal true:", (len(removal_true)))

removal_false = stop_df[stop_df["Removal of more than just outer clothing"] == False]
print("removal false:", len(removal_false))

# Spatial
print("Spatial dataset:")

link_true = stop_spatial_df[stop_spatial_df["Outcome linked to object of search"] == True]
print("link true:", len(link_true))

link_false = stop_spatial_df[stop_spatial_df["Outcome linked to object of search"] == False]
print("link false:", len(link_false))

removal_true = stop_spatial_df[stop_spatial_df["Removal of more than just outer clothing"] == True]
print("removal true:", len(removal_true))

removal_false = stop_spatial_df[stop_spatial_df["Removal of more than just outer clothing"] == False]
print("removal false:", len(removal_false))

Normal dataset:
link true: 6776
link false: 76401
removal true: 1752
removal false: 82991
Spatial dataset:
link true: 4474
link false: 67946
removal true: 1403
removal false: 71062


In [92]:
from pathlib import Path
import pandas as pd

# Ensure your output base folder exists
base_folder_normal = output_path / "stop_and_search_cleaned"
base_folder_spatial = output_path / "stop_and_search_cleaned_spatial"
base_folder_normal.mkdir(parents=True, exist_ok=True)
base_folder_spatial.mkdir(parents=True, exist_ok=True)

# Ensure 'datetime' column exists and is in datetime format
stop_df["datetime"] = pd.to_datetime(stop_df["Date"], errors='coerce')
stop_spatial_df["datetime"] = pd.to_datetime(stop_spatial_df["Date"], errors='coerce')

# Ensure timezone consistency
start_date = pd.to_datetime("2016-05-01").tz_localize(None)
end_date = min(
    stop_df["datetime"].max(),
    stop_spatial_df["datetime"].max()
).tz_localize(None)

# Generate list of months
all_months = pd.date_range(start=start_date, end=end_date, freq="MS")

for date in all_months:
    year = date.year
    month = date.month

    # Filter data for the month
    normal_month_df = stop_df[(stop_df["datetime"].dt.month == month) & (stop_df["datetime"].dt.year == year)]
    spatial_month_df = stop_spatial_df[(stop_spatial_df["datetime"].dt.month == month) & (stop_spatial_df["datetime"].dt.year == year)]

    # Format filename as "YYYY_MM-..."
    filename_normal = f"{year}-{month:02d}_stop_and_search_cleaned.csv"
    filename_spatial = f"{year}-{month:02d}_stop_and_search_cleaned_spatial.csv"

    # Save to corresponding folders
    normal_month_df.to_csv(base_folder_normal / filename_normal, index=False)
    spatial_month_df.to_csv(base_folder_spatial / filename_spatial, index=False)

    print(f"Saved: {filename_normal} and {filename_spatial}")


Saved: 2016-05_stop_and_search_cleaned.csv and 2016-05_stop_and_search_cleaned_spatial.csv
Saved: 2016-06_stop_and_search_cleaned.csv and 2016-06_stop_and_search_cleaned_spatial.csv
Saved: 2016-07_stop_and_search_cleaned.csv and 2016-07_stop_and_search_cleaned_spatial.csv
Saved: 2016-08_stop_and_search_cleaned.csv and 2016-08_stop_and_search_cleaned_spatial.csv
Saved: 2016-09_stop_and_search_cleaned.csv and 2016-09_stop_and_search_cleaned_spatial.csv
Saved: 2016-10_stop_and_search_cleaned.csv and 2016-10_stop_and_search_cleaned_spatial.csv
Saved: 2016-11_stop_and_search_cleaned.csv and 2016-11_stop_and_search_cleaned_spatial.csv
Saved: 2016-12_stop_and_search_cleaned.csv and 2016-12_stop_and_search_cleaned_spatial.csv
Saved: 2017-01_stop_and_search_cleaned.csv and 2017-01_stop_and_search_cleaned_spatial.csv
Saved: 2017-02_stop_and_search_cleaned.csv and 2017-02_stop_and_search_cleaned_spatial.csv
Saved: 2017-03_stop_and_search_cleaned.csv and 2017-03_stop_and_search_cleaned_spatial.csv

In [93]:
import pandas as pd
from pathlib import Path

# Base folder where your cleaned data folders reside (adjust if needed)
base_path = Path("C:/Users/20231737/OneDrive/Dokumentumok/Y2Q4/CBL/Other Data")

# Paths to your cleaned data folders
normal_folder = base_path / "stop_and_search_cleaned"
spatial_folder = base_path / "stop_and_search_cleaned_spatial"

# Function to load all CSVs from a folder into one DataFrame
def load_all_csvs(folder_path):
    dfs = []
    for csv_file in sorted(folder_path.glob("*.csv")):
        try:
            df = pd.read_csv(csv_file)
            dfs.append(df)
        except Exception as e:
            print(f"Failed to load {csv_file}: {e}")
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # empty DataFrame if no files

# Load normal and spatial data
hop_df = load_all_csvs(normal_folder)
hop_spatial_df = load_all_csvs(spatial_folder)

print(f"Loaded {len(stop_df)} rows from normal cleaned data")
print(f"Loaded {len(stop_spatial_df)} rows from spatial cleaned data")


Loaded 1690262 rows from normal cleaned data
Loaded 1345087 rows from spatial cleaned data
